## Problem Set 2 (Part 2 of 2): Interacting with the Alpha Vantage API using Python libraries

In Part 2 of Problem Set 2 we will be using the data retrived from the Alpha Vantage API and creating an optimized portfolio of stocks based on Modern Portfolio Theory. The PyPortfolioOpt package will be used to optimize a portfolio of stocks based maximizing the Sharpe Ratio. This exercise is an example of how a robo-advisor would use data APIs to continuously monitor and adjusts stock portfolios. 

This part of the exercise demonstrates:

1. Setting up a programming environment to work with APIs
2. Data preprocessing to create compatible datasets
3. Using a quantititive popular portfolio optimization library to estimate risks and weights

Documentation links:
<br />Alpha Vantage: https://www.alphavantage.co/documentation/
<br />Alpha Vantage Python library: https://alpha-vantage.readthedocs.io/en/latest/#
<br />Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
<br />PyPortfolioOpt: https://pyportfolioopt.readthedocs.io/en/latest/index.html

In [1]:
api_key = "AUDS4B0HY9KIO68B"

In [2]:
import pandas as pd
import requests
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

# PyPortfolioOpt libraries
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [3]:
ts = TimeSeries(key=api_key,output_format='pandas')

In [5]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [6]:
print(fb_data)

            1. open  2. high  3. low  4. close    5. volume
date                                                       
2020-10-21  279.560   283.05  276.37  278.7300   28892712.0
2020-10-20  263.060   269.70  262.88  267.5600   18763246.0
2020-10-19  265.530   268.55  259.88  261.4000   13586955.0
2020-10-16  267.375   271.37  265.30  265.9300   16622702.0
2020-10-15  267.600   269.04  263.67  266.7200   15416064.0
...             ...      ...     ...       ...          ...
2012-05-24   32.950    33.21   31.77   33.0300   50237200.0
2012-05-23   31.370    32.50   31.36   32.0000   73600000.0
2012-05-22   32.610    33.59   30.94   31.0000  101786600.0
2012-05-21   36.530    36.66   33.00   34.0300  168192700.0
2012-05-18   42.050    45.00   38.00   38.2318  573576400.0

[2121 rows x 5 columns]


Isolate "close" column of data for each security:

In [7]:
fb_close = fb_data["4. close"]
msft_close = msft_data["4. close"]
aapl_close = aapl_data["4. close"]
amzn_close = amzn_data["4. close"]

Create new dataframe with tickers as columns:

In [8]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

Place data into new dataframe:

In [9]:
df['fb'] = fb_close
df['msft'] = msft_close
df['aapl'] = aapl_close
df['amzn'] = amzn_close

In [10]:
print(df)

                  fb    msft    aapl     amzn
date                                         
2020-10-21  278.7300  214.80  116.87  3184.94
2020-10-20  267.5600  214.65  117.51  3217.01
2020-10-19  261.4000  214.22  115.98  3207.21
2020-10-16  265.9300  219.66  119.02  3272.71
2020-10-15  266.7200  219.66  120.71  3338.65
...              ...     ...     ...      ...
2012-05-24   33.0300   29.07  565.32   215.24
2012-05-23   32.0000   29.11  570.56   217.28
2012-05-22   31.0000   29.76  556.97   215.33
2012-05-21   34.0300   29.75  561.28   218.11
2012-05-18   38.2318   29.27  530.38   213.85

[2121 rows x 4 columns]


Take only the earlist half of observations:

In [11]:
df_fh = df[0:2121]

In [12]:
df_fh.count()

fb      2121
msft    2121
aapl    2121
amzn    2121
dtype: int64

Calculate expected returns:

In [13]:
mu = expected_returns.mean_historical_return(df_fh)

In [14]:
print(mu)

fb     -0.210330
msft   -0.210946
aapl    0.196968
amzn   -0.274614
dtype: float64


Calculate sample covariance matrix:

In [15]:
S = risk_models.sample_cov(df_fh)

In [16]:
print(S)

            fb      msft      aapl      amzn
fb    0.137102  0.041021  0.038861  0.050464
msft  0.041021  0.068983  0.052412  0.043248
aapl  0.038861  0.052412  5.183581  0.039443
amzn  0.050464  0.043248  0.039443  0.091101


Optimize the weights to maximzie the Sharpe Ratio:

In [17]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 19.7%
Annual volatility: 227.7%
Sharpe Ratio: 0.08


(0.19696774914127135, 2.276747867067293, 0.0777283034722794)

In [18]:
print(weights)

OrderedDict([('fb', 0.0), ('msft', 0.0), ('aapl', 1.0), ('amzn', 0.0)])


Using optimized weights, compute portfolio accolation:

In [19]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

{'aapl': 1885}


To update portfolio, call the API again for latest data and run optimization model again:

In [20]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [21]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

In [22]:
df['fb'] = fb_close
df['msft'] = msft_close
df['aapl'] = aapl_close
df['amzn'] = amzn_close
print(df)

                  fb    msft    aapl     amzn
date                                         
2020-10-21  278.7300  214.80  116.87  3184.94
2020-10-20  267.5600  214.65  117.51  3217.01
2020-10-19  261.4000  214.22  115.98  3207.21
2020-10-16  265.9300  219.66  119.02  3272.71
2020-10-15  266.7200  219.66  120.71  3338.65
...              ...     ...     ...      ...
2012-05-24   33.0300   29.07  565.32   215.24
2012-05-23   32.0000   29.11  570.56   217.28
2012-05-22   31.0000   29.76  556.97   215.33
2012-05-21   34.0300   29.75  561.28   218.11
2012-05-18   38.2318   29.27  530.38   213.85

[2121 rows x 4 columns]


In [23]:
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

{'aapl': 1885}
